# 신경망 하이퍼파라미터 튜닝 방법
`하이퍼파라미터`란, 개발자가 직접 조절해주는 변수로, 모델 성능에 큰 영향을 미친다.

In [ ]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 커스텀 라이브러리 임포트
from func_debug_log import debug

In [ ]:
# debug_log (0 : 미출력, 1 : 함수 실행 로그 출력, 2 : 함수 내부)
debug_flag = 1

## 1. 교차 검증 적용하기

In [ ]:
if debug_flag in range(1, 3) : debug('신경망에 교차 검증 적용하기', __name__)

In [ ]:
if debug_flag in range(1, 3) : debug('라이브러리 임포트', __name__)

In [ ]:
if debug_flag in range(1, 3) : debug('데이터 불러오기', __name__)

In [ ]:
if debug_flag in range(1, 3) : debug('Kfold 설정', __name__)

## 2. 가중치 초기화 (Weight Initialization)

### [1] 가중치 초기화란?
초기 가중치를 최적점에 가깝게 설정하는 방법

### [2] 가중치 초기화의 종류

#### <1> 표준편차가 1인 정규분포
+ 대부분의 활성화 값이 0 or 1 -> 학습이 정상적으로 이루어 지지 않음

#### <2> Xavier-Glorot 초기화
+ 이전층의 노드 수가 $n$ 일 때, 현재층의 가중치를 표준편차가 ${1 \over \sqrt{n}}$인 정규분포로 초기화
+ 표준편차가 고정된 경우 학습이 정상적으로 이루어지지 않는 점을 해결.(n은 이전과 현재 층의 노드를 같이 쓰는 등 변화 가능)
+ Sigmoid 활성화 함수에서는 잘 작동하나, ReLU 활성화 함수에서는 층이 지날수록, 활성화 값이 편중됨. -> 학습이 정상적으로 이루어 지지 않음

#### <3> He 초기화
+ 이전층의 노드 수가 $n$ 일 때, 현재층의 가중치를 표준편차가 ${2 \over \sqrt{n}}$인 정규분포로 초기화
+ ReLU를 사용할 경우 층이 깊어질수록 학습이 정상적으로 이루어지지 않는 점을 해결.

### [3] 초기화 방법
```
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
```

## 3. 과적합 (Overfitting)
train 데이터에 과하게 최적화되어, 일반적인 성능이 떨어지는 현상을 말하며, 층이 증가함에 따라 매개 변수가 기하급수적으로 증가하는 인공지능에서는 더 쉽게 발생함.

### [1] 가중치 감소 (Weight Decay)
+ 손실 함수에 가중치 관련항을 넣어, 가중치가 너무 커지는 것을 방지.
+ L1 Regularization(LASSO) : $L_1(\theta_w) = {1 \over 2}\sum_i(output_i - target_i)^2 + \lambda \cdot ||\theta_w||_1$
+ L2 Regularization(Ridge) : $L_2(\theta_w) = {1 \over 2}\sum_i(output_i - target_i)^2 + \lambda \cdot ||\theta_w||_2$
```
Dense(64,
      kernel_regularizer=regularizers.l2(0.01),
      activity_regularizer=regularizers.l1(0.01))
```

### [2] Dropout 
+ Iteration 마다 각 노드가 일정 확률로 사용되지 않도록 한다.
```
Dense(64,
      kernel_regularizer=regularizers.l2(0.01),
      activity_regularizer=regularizers.l1(0.01))
Dropout(0.5)
```

### [3] 조기 종료 (Early Stopping)
+ 학습이 계속되면서, Train 데이터에 대한 손실은 줄어드는데, Validation 데이터에 대한 손실이 증가하면(과적합이 발생하기 시작하면) 학습을 종료하는 방법

# Fashion MNIST 예제

In [ ]:
# 라이브러리 임포트
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers

In [ ]:
if debug_flag in range(1, 3) : debug('seed 설정', __name__)
np.random.seed(42)
tf.random.set_seed(42)

2022.12.02 18:11:39 __main__ seed 설정


In [ ]:
if debug_flag in range(1, 3) : debug('데이터 불러오기', __name__)
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

2022.12.02 18:11:39 __main__ 데이터 불러오기


In [ ]:
if debug_flag in range(1, 3) : debug('데이터 정규화', __name__)
X_train = X_train / 255.
X_test = X_test / 255.

2022.12.02 18:11:39 __main__ 데이터 정규화


In [ ]:
if debug_flag in range(2, 3) : debug('레이블 개수와 형태 확인', __name__)
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
if debug_flag in range(1, 3) : debug('신경망 구축', __name__)
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64,
          kernel_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l1(0.01)),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

2022.12.02 18:11:39 __main__ 신경망 구축


In [ ]:
if debug_flag in range(1, 3) : debug('신경망 컴파일', __name__)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.89)
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])

2022.12.02 18:11:39 __main__ 신경망 컴파일


In [ ]:
if debug_flag in range(2, 3) : debug('신경망 요약 출력', __name__)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [ ]:
if debug_flag in range(2, 3) : debug('파라미터 저장 경로 설정', __name__)
checkpoint_filepath = "FMbest.hdf5"

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)


In [ ]:
if debug_flag in range(2, 3) : debug('Callback 함수 설정', __name__)
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)


In [ ]:
if debug_flag in range(1, 3) : debug('신경망 학습', __name__)
model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, 
          validation_data=(X_test,y_test), 
          callbacks=[early_stop, save_best])

2022.12.02 18:11:39 __main__ 신경망 학습
Epoch 1/30
1870/1875 [============================>.] - ETA: 0s - loss: 1.2128 - accuracy: 0.7681
Epoch 1: val_loss improved from inf to 0.86062, saving model to FMbest.hdf5
1875/1875 [==============================] - 10s 5ms/step - loss: 1.2121 - accuracy: 0.7682 - val_loss: 0.8606 - val_accuracy: 0.8135
Epoch 2/30
1866/1875 [============================>.] - ETA: 0s - loss: 0.9309 - accuracy: 0.7882
Epoch 2: val_loss did not improve from 0.86062
1875/1875 [==============================] - 9s 5ms/step - loss: 0.9306 - accuracy: 0.7884 - val_loss: 0.8890 - val_accuracy: 0.7957
Epoch 3/30
1871/1875 [============================>.] - ETA: 0s - loss: 0.9075 - accuracy: 0.7917
Epoch 3: val_loss improved from 0.86062 to 0.85530, saving model to FMbest.hdf5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.9084 - accuracy: 0.7915 - val_loss: 0.8553 - val_accuracy: 0.7967
Epoch 4/30
1872/1875 [============================>.] - ETA: 0s - l

In [ ]:
if debug_flag in range(1, 3) : debug('조기 종료 직전 모델 평가', __name__)

2022.12.02 18:11:39 __main__ 조기 종료 직전 모델 평가


In [ ]:
if debug_flag in range(2, 3) : debug('모델 예측 수행', __name__)
model.predict(X_test[0:1])

1/1 [==============================] - 0s 104ms/step


array([[2.2530679e-05, 1.4091254e-06, 5.0333696e-05, 4.7060814e-05,
        8.4856219e-05, 1.5336972e-01, 9.2422488e-05, 2.5663882e-01,
        3.0773242e-03, 5.8661544e-01]], dtype=float32)

In [ ]:
if debug_flag in range(2, 3) : debug('모델 평가', __name__)
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.8543 - accuracy: 0.8051 - 684ms/epoch - 2ms/step


In [ ]:
if debug_flag in range(1, 3) : debug('Callback 모델 확인 및 평가', __name__)

2022.12.02 18:11:39 __main__ Callback 모델 확인 및 평가


In [ ]:
if debug_flag in range(2, 3) : debug('모델 불러오기', __name__)
model.load_weights(checkpoint_filepath)

In [ ]:
if debug_flag in range(2, 3) : debug('모델 예측 수행', __name__)
model.predict(X_test[0:1])

1/1 [==============================] - 0s 23ms/step


array([[3.4490717e-05, 4.8838134e-05, 6.7998852e-05, 1.3713697e-04,
        6.0494600e-05, 2.1046488e-01, 8.1299098e-05, 1.3274866e-01,
        2.1887003e-03, 6.5416753e-01]], dtype=float32)

In [ ]:
if debug_flag in range(2, 3) : debug('모델 평가', __name__)
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

313/313 [==============================] - 1s 3ms/step - loss: 0.7896 - accuracy: 0.8198
